In [30]:
import re
import pandas as pd

import numpy as np

from tqdm import tqdm
import os

tqdm.pandas(desc='pandas bar')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.3f' % x)

%matplotlib inline

data_dir_s1 = './DATA/s1/'
data_dir_s2 = './DATA/s2/'
data_dir_s3 = './DATA/s3/'

data_dir_t1 = './DATA/t1/'
data_dir_t2 = './DATA/t2/'

In [31]:

# # source市场
# train_s1 = pd.read_table(data_dir_s1+'train.tsv')
# train_s2 = pd.read_table(data_dir_s2+'train.tsv')
# train_s3 = pd.read_table(data_dir_s3+'train.tsv')

# train_core_s1 = pd.read_table(data_dir_s1+'train_5core.tsv')
# train_core_s2 = pd.read_table(data_dir_s2+'train_5core.tsv')
# train_core_s3 = pd.read_table(data_dir_s3+'train_5core.tsv')

# target市场
train_coret1 = pd.read_table(data_dir_t1+'train_5core.tsv')
train_t1 = pd.read_table(data_dir_t1+'train.tsv')
vaild_qrel_t1 = pd.read_table(data_dir_t1+'valid_qrel.tsv')
vaild_pred_t1 = pd.read_table('/ljw/wsdm22_cup_xmrec/wsdm22_cup_xmrec_rate/baseline_outputs/sample_run/t1/valid_pred.tsv')
test_t1 = pd.read_table(data_dir_t1+'test_run.tsv')

train_coret2 = pd.read_table(data_dir_t2+'train_5core.tsv')
train_t2 = pd.read_table(data_dir_t2+'train.tsv')
vaild_qrel_t2 = pd.read_table(data_dir_t2+'valid_qrel.tsv')
vaild_pred_t2 = pd.read_table('/ljw/wsdm22_cup_xmrec/wsdm22_cup_xmrec_rate/baseline_outputs/sample_run/t2/valid_pred.tsv')

In [32]:
train_t1

,userId,itemId,rating
0,t1U1006129,P1016853,5.000
1,t1U1002135,P1019605,5.000
2,t1U1006710,P1032538,5.000
3,t1U1007418,P1012632,5.000
4,t1U1008062,P1022775,5.000
...,...,...,...
60163,t1U1001650,P1007676,5.000
60164,t1U1008860,P1003805,4.000
60165,t1U1004239,P1001881,5.000
60166,t1U1002518,P1001155,5.000


In [33]:
# t1
vaild = vaild_pred_t1.merge(vaild_qrel_t1, on=['userId', 'itemId'], how='left')
vaild = vaild.groupby("userId").agg(
    score_list=("score", list),
    rating_list=("rating", list)
).reset_index()

# 找到预测的位置
vaild['posi_pos'] = vaild['rating_list'].apply(lambda x:x.index(1.0)+1)
print('t1  all: {},pos of 1st: {},pos of >10: {}'.format(vaild.shape[0], vaild[vaild['posi_pos']==1].shape[0], vaild[vaild['posi_pos']>10].shape[0]))


# t2
vaild = vaild_pred_t2.merge(vaild_qrel_t2, on=['userId', 'itemId'], how='left')
vaild = vaild.groupby("userId").agg(
    score_list=("score", list),
    rating_list=("rating", list)
).reset_index()

# 找到预测的位置
vaild['posi_pos'] = vaild['rating_list'].apply(lambda x:x.index(1.0)+1)
print('t2  all: {},pos of 1st: {},pos of >10: {}'.format(vaild.shape[0], vaild[vaild['posi_pos']==1].shape[0], vaild[vaild['posi_pos']>10].shape[0]))

t1  all: 2697,pos of 1st: 1665,pos of >10: 614
t2  all: 5482,pos of 1st: 2826,pos of >10: 1669


In [38]:
vaild[vaild['posi_pos']<=10].head(200)

,userId,score_list,rating_list,posi_pos
0,t2U1000006,"[0.9868181938058932, 0.1676321095366815, 0.122...","[1.0, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
1,t2U1000008,"[0.0357748828253237, 0.0292453009967602, 0.025...","[nan, 1.0, nan, nan, nan, nan, nan, nan, nan, ...",2
3,t2U1000013,"[0.154613720701066, 0.0621113256840447, 0.0352...","[nan, 1.0, nan, nan, nan, nan, nan, nan, nan, ...",2
4,t2U1000014,"[0.0682251477088137, 0.0343719561090024, 0.024...","[nan, nan, nan, nan, nan, nan, nan, nan, 1.0, ...",9
5,t2U1000015,"[0.9985441159380604, 0.1062961830947056, 0.025...","[1.0, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
7,t2U1000018,"[0.8649574281525276, 0.0352428302875294, 0.032...","[1.0, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
8,t2U1000020,"[0.0678359253853435, 0.0646900484086737, 0.054...","[nan, nan, nan, nan, nan, 1.0, nan, nan, nan, ...",6
9,t2U1000021,"[0.0698317878957367, 0.0534591007556523, 0.026...","[nan, nan, nan, nan, nan, 1.0, nan, nan, nan, ...",6
10,t2U1000026,"[0.9975714019825064, 0.0503839329992428, 0.033...","[1.0, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
11,t2U1000032,"[0.9979827445443056, 0.0079131442011942, 0.007...","[1.0, nan, nan, nan, nan, nan, nan, nan, nan, ...",1
